# Задача

ДЗ делаем по бд orders
В качестве ДЗ сделаем карту поведения пользователей. Мы обсуждали, что всех пользователей можно разделить, к примеру, на New (совершили только 1 покупку), Regular (совершили 2 или более на сумму не более стольки-то), Vip (совершили дорогие покупки и достаточно часто), Lost (раньше покупали хотя бы раз и с даты последней покупки прошло больше 3 месяцев). Вся база должна войти в эти гурппы (т.е. каждый пользователь должен попадать только в одну из этих групп).

**Задача**:
1. Уточнить критерии групп New,Regular,Vip,Lost
2. По состоянию на 1.01.2017 понимаем, кто попадает в какую группу, подсчитываем кол-во пользователей в каждой.
3. По состоянию на 1.02.2017 понимаем, кто вышел из каждой из групп, а кто вошел.
4. Аналогично смотрим состояние на 1.03.2017, понимаем кто вышел из каждой из групп, а кто вошел.
5. В итоге делаем вывод, какая группа уменьшается, какая увеличивается и продумываем, в чем может быть причина.

Присылайте отчет в pdf

# Подготовка

## Подключение библиотек

In [1]:
import pymysql.cursors
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import getpass

## Подключение к базе данных

In [2]:
# password_mysql = getpass.getpass(prompt="Введите пароль от MySQL-сервера: ")
password_mysql = "pass123"
mydb = pymysql.connect(host="127.0.0.1",
                             user='root',
                             password=password_mysql, 
                             db='db_analyt_loc',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
print ("connect successful!!")

connect successful!!


In [3]:
def show_tables():
    return pd.read_sql("""SHOW TABLES FROM db_analyt_loc;""", con=mydb)
show_tables()

,Tables_in_db_analyt_loc
0,orders_20190822
1,orders_all
2,time_user_b
3,time_user_ym_s
4,users_for_merg


1. Уточнить критерии групп New,Regular,Vip,Lost

In [ ]:
%%time


2. По состоянию на 1.01.2017 понимаем, кто попадает в какую группу, подсчитываем кол-во пользователей в каждой.

3. По состоянию на 1.02.2017 понимаем, кто вышел из каждой из групп, а кто вошел.

4. Аналогично смотрим состояние на 1.03.2017, понимаем кто вышел из каждой из групп, а кто вошел.

5. В итоге делаем вывод, какая группа уменьшается, какая увеличивается и продумываем, в чем может быть причина.

### 1. На 2016.12.31 получаем инфу по кол-ву заказов на каждого юзера. Подсчитываем сколько пользователей попало в группу с 1 заказом, 2, 3 и т.д.

In [ ]:
%%time
# Второй раз выполнять не надо
mydb.cursor().execute(
    """
    CREATE TABLE time_user_b
        SELECT
          user_id,
          COUNT(id_o) n,
          DATE('2017-01-01') m
        FROM orders_20190822 o
        WHERE DATE(o_date) < DATE('2017-01-01')
        GROUP BY user_id;
    """)
mydb.commit()

months = ['02','03','04','05','06','07','08','09','10','11','12']
for month in months:
    mydb.cursor().execute(
        f"""
        INSERT INTO time_user_b
            SELECT
            user_id,
            COUNT(id_o) n,
            DATE('2017-{month}-01')
            FROM orders_20190822 o
            WHERE DATE(o_date) < DATE('2017-{month}-01')
            GROUP BY user_id;
          """)
    mydb.commit()

In [ ]:
show_tables()

In [ ]:
mydb.cursor().execute(
    """
    CREATE TABLE time_user_ym_s
        SELECT
          user_id,
          DATE_FORMAT(o_date, '%y%m') ym,
          SUM(price) s
        FROM orders_20190822 o
        GROUP BY user_id, DATE_FORMAT(o_date, '%y%m');
    """)
mydb.commit()
show_tables()

In [ ]:
%%time
mydb.cursor().execute(
    """
    CREATE TABLE time_user_ym_s
    SELECT
      user_id,
      DATE_FORMAT(o_date, '%y%m') AS ym,
      SUM(price) AS s
    FROM orders_20190822 o
    GROUP BY user_id, DATE_FORMAT(o_date, '%y%m');
    """
)
mydb.commit()

In [ ]:
pd.read_sql(f"SELECT * FROM time_user_ym_s", con=mydb)

# Закрытие сессии

In [ ]:
mydb.close()